### Get Detailed Information on Shoes to Sell with ChatGPT

In [ ]:
shoe_str = "DV by dolche vida size 11 Elaine in Rose Gold"
# shoe_str = "Adidas Samba Classic Black White Mens Size 12"
# shoe_str = "Birkenstock sandals bostonian size 9 womens"

In [ ]:
import json
import openai
import os
import requests
import dotenv
from openai import OpenAI, OpenAIError, AsyncOpenAI

In [ ]:
dotenv.load_dotenv()

In [ ]:
GPT_MODEL = "gpt-4"
openai.api_key = os.environ['OPENAI_API_KEY']

client = openai.OpenAI()

def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


### Create a Queue Entry with the information

In [15]:
shoe = chat_completion_request([
    {"role": "system", "content": "You are a precise data extractor. Respond only with the requested JSON format."},
    {"role": "user", "content": f"{shoe_str}"},])

### Parse the Response

In [16]:
json.loads(shoe.choices[0].message.content)

{'Brand': 'DV by dolche vida',
 'Size': '11',
 'Model': 'Elaine',
 'Color': 'Rose Gold'}

In [17]:
def chat_enhance_shoe_data(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [18]:
#print(shoe.choices[0].message.content)
shoe_data = json.loads(shoe.choices[0].message.content)

In [19]:
# get shoe prompt
with open('static/prompts/shoe_prompt.txt', 'r') as file: shoe_prompt = file.read()

In [20]:
def clean_json_blocks(text: str) -> str:
    """
    Checks if text starts with ```json and ends with ``` and removes these markers if present.
    Properly handles newline characters after the opening marker.
    
    Args:
        text (str): The input text to process
        
    Returns:
        str: Text with JSON code block markers removed if they existed, otherwise original text
    """
    # Strip whitespace to handle cases with leading/trailing spaces
    stripped_text = text.strip()
    
    # Check if text starts and ends with the markers
    starts_with_json = stripped_text.startswith("```json")
    ends_with_backticks = stripped_text.endswith("```")
    
    if starts_with_json and ends_with_backticks:
        # Remove the ```json marker and any single following newline
        content = stripped_text[6:]  # Remove ```json
        if content.startswith('\n'):
            content = content[1:]  # Remove single newline after ```json
            
        # Remove the ending ```
        if content.endswith('```'):
            content = content[:-3]
            
        return content.strip()
    
    return text

In [21]:
enhanced_shoe_data = chat_completion_request([
    {"role": "system", "content": f"{shoe_prompt} DO NOT RETURN ANY TEXT OR COMMENTS ONLY JSON DATA"},
    {"role": "user", "content": f"{shoe_data}"}]
)

In [22]:
# enhance the existing data on the shoes
output = enhanced_shoe_data.choices[0].message.content
output = clean_json_blocks(output)
output = json.loads(output)

In [23]:
# print the enhanced shoe data
output

{'brand': 'DV by dolche vida',
 'model': 'Elaine',
 'size': '11',
 'gender': 'Womens',
 'color': 'Rose Gold',
 'condition': 'New',
 'shoe_type': 'Heels',
 'style': 'Ballet',
 'material': 'Leather',
 'fit_type': 'Regular',
 'pattern': 'Solid',
 'occasion': 'Casual',
 'season': 'All Season',
 'length': None,
 'upc': '884095273938',
 'msrp': 45.0,
 'asking_price': 35.0,
 'sale_price': None,
 'description': "DV by dolche vida Women's Elaine Ballet Flat Heels, Size 11, Rose Gold.",
 'quantity': 1,
 'condition_id': 1000,
 'dispatch_time': 1,
 'return_accepted': True,
 'return_policy': '30-day returns',
 'shipping_type': 'Flat',
 'shipping_cost': 0.0,
 'international_shipping': False,
 'item_location': 'New York, NY',
 'listing_status': 'Active',
 'views': 0,
 'watchers': 0,
 'last_updated': '2022-03-30T18:24:41.379Z',
 'created_at': '2022-03-30T18:24:41.379Z'}

In [4]:
import os
print("User Token", os.getenv('CURRENT_USER_TOKEN'))
print("User ID", os.getenv('CURRENT_USER_TOKEN'))

User Token None
User ID None


In [25]:
print(output['gender'])
output['gender'] = output['gender'].replace("'", "")
print(output['gender'])

Womens
Womens


In [ ]:
url = "https://api.denkers.co/shoes/" 

headers = {
    "x-token": os.getenv('CURRENT_USER_TOKEN'),
    "user_id": os.getenv('CURRENT_USER_ID'),
    "Content-Type": "application/json",
    "Accept": "application/json",
}


In [32]:
print(headers)
print(output)

{'x-token': '6b1f9eeb7e3a1f6e3b3f1a2c5a7d9e1b', 'user_id': '1', 'Content-Type': 'application/json', 'Accept': 'application/json'}
{'brand': 'DV by dolche vida', 'model': 'Elaine', 'size': '11', 'gender': 'Womens', 'color': 'Rose Gold', 'condition': 'New', 'shoe_type': 'Heels', 'style': 'Ballet', 'material': 'Leather', 'fit_type': 'Regular', 'pattern': 'Solid', 'occasion': 'Casual', 'season': 'All Season', 'length': None, 'upc': '884095273938', 'msrp': 45.0, 'asking_price': 35.0, 'sale_price': None, 'description': "DV by dolche vida Women's Elaine Ballet Flat Heels, Size 11, Rose Gold.", 'quantity': 1, 'condition_id': 1000, 'dispatch_time': 1, 'return_accepted': True, 'return_policy': '30-day returns', 'shipping_type': 'Flat', 'shipping_cost': 0.0, 'international_shipping': False, 'item_location': 'New York, NY', 'listing_status': 'Active', 'views': 0, 'watchers': 0, 'last_updated': '2022-03-30T18:24:41.379Z', 'created_at': '2022-03-30T18:24:41.379Z'}


In [35]:
print(json.dumps(output))

{"brand": "DV by dolche vida", "model": "Elaine", "size": "11", "gender": "Womens", "color": "Rose Gold", "condition": "New", "shoe_type": "Heels", "style": "Ballet", "material": "Leather", "fit_type": "Regular", "pattern": "Solid", "occasion": "Casual", "season": "All Season", "length": null, "upc": "884095273938", "msrp": 45.0, "asking_price": 35.0, "sale_price": null, "description": "DV by dolche vida Women's Elaine Ballet Flat Heels, Size 11, Rose Gold.", "quantity": 1, "condition_id": 1000, "dispatch_time": 1, "return_accepted": true, "return_policy": "30-day returns", "shipping_type": "Flat", "shipping_cost": 0.0, "international_shipping": false, "item_location": "New York, NY", "listing_status": "Active", "views": 0, "watchers": 0, "last_updated": "2022-03-30T18:24:41.379Z", "created_at": "2022-03-30T18:24:41.379Z"}


#### Execute the Request

In [36]:
params = {
    "user_id": "1"
}
response = requests.post(url, headers=headers, json=output, params=params)

#### What Happened

In [37]:
print(response.status_code)
print(response.content)

500
b'{"detail":"Database error occurred"}'


In [97]:
print(json.dumps(output))

{"brand": "DV by Dolce Vita", "model": "Elaine", "size": "11", "gender": "Womens", "color": "Rose Gold", "condition": "New", "shoe_type": "Sandals", "style": "Strappy Sandals", "material": "Synthetic", "fit_type": "Standard Fit", "pattern": "Solid", "occasion": "Casual, Outdoor", "season": "Spring, Summer", "length": null, "upc": "888133729889", "msrp": 35.99, "asking_price": 29.99, "sale_price": null, "description": "Stand out in style with the DV by Dolce Vita Elaine sandals in Rose Gold. They are designed to leave a lasting impression wherever you go. They offer a comfortable fit and a stylish design that is perfect for any occasion.", "quantity": 1, "condition_id": 1000, "dispatch_time": 1, "return_accepted": true, "return_policy": "30-day returns", "shipping_type": "Flat", "shipping_cost": 0.0, "international_shipping": false, "item_location": "Los Angeles, CA", "listing_status": "Active", "views": 0, "watchers": 0, "last_updated": "2022-09-28T12:34:56Z", "created_at": "2022-09-28